In [22]:
import pandas as pd

# Custom
from processing import tag_utterances
from processing import load_sem_types
from processing import load_comments
pd.set_option('display.max_columns', 500) # more columns displayed at once

In [23]:
# import comments
path_to_data = '../data/reddit_comments_askDocs_2014_to_2018_03.gz'
df = load_comments(path_to_data)

Shape: (557648, 23)


There is a lot that could be done with formatting the data for training conversations:

* Option 1: All responses are equal
    * Treat every thread as a conversation
    * Every comment in the thread as a response to the original AskDocs

## Option 1: All responses are equal

3 files to create:
* Thread conversations: Contains dialogue structure
* Thread lines: Contains actual text of each utterance
* Utterance metadata: Contains information about each utterance such as if the author was a clinician,moderator, data, score etc.

In [20]:
list_of_threads = df['link_id_short'].unique().tolist()

# loop through all threads
for thread in list_of_threads[:3]:
    df_thread = df.loc[df['link_id_short'] == thread
    # get authors
    

In [21]:
df_thread.head()

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,removal_reason,link_id_short,parent_id_short
2,Never been tested for that. I was hoping the ...,1.0,NaN,NaN,-o2,This user has not yet been verified.,NaN,1461952470,t5_2xtuc,t3_4gz1fi,t1_d2mbnhi,1,1.463777e+09,0,0,d2mce34,AskDocs,1.0,NaN,default,NaN,4gz1fi,d2mbnhi
50215,Have you ever been diagnosed or treated for GE...,1.0,NaN,NaN,cuteman,This user has not yet been verified.,NaN,1461951490,t5_2xtuc,t3_4gz1fi,t3_4gz1fi,2,1.463777e+09,0,0,d2mbnhi,AskDocs,2.0,NaN,default,NaN,4gz1fi,4gz1fi
50338,Chest pain (angina) can often be a manifestati...,1.0,NaN,NaN,cuteman,This user has not yet been verified.,NaN,1461955247,t5_2xtuc,t3_4gz1fi,t1_d2mce34,2,1.463778e+09,0,0,d2meh7d,AskDocs,2.0,NaN,default,NaN,4gz1fi,d2mce34


In [15]:
df.distinguished.value_counts()

moderator    16938
Name: distinguished, dtype: int64